In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://securit

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 22:06:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.66MB/s    in 0.2s    

2022-11-30 22:06:40 (5.66 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"),sep="\t",header=True,inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [5]:
# Get the number of rows in the DataFrame.
df.count()

145431

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = df.select(["review_id","customer_id","product_id","product_parent",to_date(df["review_date"])])
review_id_df.show()

+--------------+-----------+----------+--------------+--------------------+
|     review_id|customer_id|product_id|product_parent|to_date(review_date)|
+--------------+-----------+----------+--------------+--------------------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193|          2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535|          2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344|          2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204|          2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561|          2015-08-31|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192|          2015-08-31|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395|          2015-08-31|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568|          2015-08-31|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568|          2015-08-31|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568|          2015-08-31|
|R3B3UHK1FO0

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"]).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.select("customer_id").groupby("customer_id").count().withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
|    3953141|             2|
|   20598424|             1|
|   46625306|             1|
|   47160828|             2|
|    1337034|             1|
|   14231016|             1|
|   49785748|             1|
|     712525|             1|
|   35339694|             1|
|    8211293|             1|
|   41832251|             1|
|    9037272|             1|
|   52036002|             1|
|   49514578|             1|
|   22875949|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSH1OZ87OYK92|          2|            2|          3|   N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|
| R3YOOS71KM5M9|          5|            0|          0|   N|
|R3R14UATT3OUFU|          5|            0|          0|   N|
| RV2W9SGDNQA2C|          5|            0|          0|   N|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|
|R1LRYU1V0T3O38|          4|            0|          0|   N|
| R44QKV6FE5CJ2|          5|            0|          0|   N|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|
|R1JEEW4C6R89BA|          5|            0|          0|   N|
|R3B3UHK1FO0ERS|          1|            0|          0|   N|
|R2GVSDHW513SS1|          5|            0|          0|   N|
|R1R1NT516PYT73|          5|            0|          0|   N|
|R3K624QDQKENN9|          4|            

# Load

In [11]:
#make sure you include your own endpoint, username, and password into the following strings
#I do not include my own for obvious reasons
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id", mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine", mode=mode, properties=config)